In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import geopandas as gpd
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
import rioxarray as rxr
from datetime import datetime
import glob
import os
import contextily as ctx
import xarray
from collections import Counter
import cmcrameri.cm as cmc
import shutil

from working_sca_funcs import *

In [12]:
# Define year you are working on
year = '2023'

# Define basin you are working on
name = 'DLNY'
# name = 'BUDD'

# Define model you are working on
# model = 'V5' # 2022 BUDD ASO 2 class
# model = 'V7' # 2022 BUDD manual classification snow/no snow trained on set of images and artifacts on different images
model = 'V9'

# raw PS images
ps_raw = f'/data0/images/planet/emma/planet/{name}/'
# ps_raw_subdir = sorted([d for d in glob.glob(ps_raw + str(year) + '*') if os.path.isdir(d)])
ps_raw_subdir = sorted([d for d in glob.glob(ps_raw + '*') if os.path.isdir(d)])

# SCA directory
ps_sca_dir = f'/home/etboud/projects/data/rerun/{name}/{model}/'
# ps_sca_tif = glob.glob(ps_sca_dir + f'*{year}*.tif')
ps_sca_tif = glob.glob(ps_sca_dir + '*.tif')

# ASO directory
aso_dir = f'/home/etboud/projects/data/aso/validation/{name}/V2/'
# aso_tif = glob.glob(os.path.join(aso_dir, f'*{year}*3m*.tif')) 
aso_tif = glob.glob(os.path.join(aso_dir, '*3m*.tif'))

# Create basin shapefile
BS = f'/home/etboud/projects/data/basins/{name}/{name}_4326.geojson'
basin = gpd.read_file(BS)
basin = basin.to_crs('EPSG:32611') 

# Extract dates from file names
ps_dates = extract_dates([], ps_sca_tif)[1]  
aso_dates = extract_dates(aso_tif, [])[0]

# canopy height model
CHM = '/home/etboud/projects/data/CHM/USCATB20140827_chm_3p0m.tif'

### Move scenes around

In [59]:
# Run for each Basin to create list of scenes to keep

dir =  glob.glob(f'/home/etboud/projects/data/rerun/{name}/V5/'+'*.tif')
save_good = []
for file in dir:
    fn = os.path.basename(file)
    save_good.append(fn)

In [62]:
# Move processed bad files to a new directory
other_model = 'V9'
path = f'/home/etboud/projects/data/rerun/{name}/{other_model}/'
move_to = f'/home/etboud/projects/data/rerun/{name}/{other_model}/not_good/'
dir = glob.glob(path + '/*.tif')
for file in dir:
    if os.path.basename(file) not in save_good:
        print(file)
        src = path+os.path.basename(file)
        dst = move_to+os.path.basename(file)
        shutil.move(src, dst)

/home/etboud/projects/data/rerun/DLNY/V9/20220829_174345_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230405_174458_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20220924_181922_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230621_174946_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230928_182711_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230823_175403_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230829_183116_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230710_182903_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230912_175511_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230402_182234_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20220601_182009_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230723_175313_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20220923_174646_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230416_182322_SCA.tif
/home/etboud/projects/data/rerun/DLNY/V9/20230515_182302_SCA.tif
/home/etboud/projects/dat

In [3]:
print(len(glob.glob('/home/etboud/projects/data/rerun/BUDD/V5' + '/*2022*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/BUDD/V9' + '/*2022*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/BUDD/V7' + '/*2022*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/BUDD/V5' + '/*2023*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/BUDD/V9' + '/*2023*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/BUDD/V7' + '/*2023*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/DLNY/V5' + '/*2023*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/DLNY/V9' + '/*2023*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/DLNY/V7' + '/*2023*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/DLNY/V7' + '/*2022*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/DLNY/V5' + '/*2022*.tif')))
print(len(glob.glob('/home/etboud/projects/data/rerun/DLNY/V9' + '/*2022*.tif')))

82
82
82
69
69
69
77
77
77
92
92
92


## Looking at NOT good files

In [37]:
# Run for each Basin to create a list of PS scenes that are bad
# Run this when model is set to V1 (unobs scenes already in not_good model)
dir =  glob.glob(f'/home/etboud/projects/data/rerun/{name}/V5/not_good/'+'*.tif')
save_bad = []
for file in dir:
    fn = os.path.basename(file)
    save_bad.append(fn)
        
# for file in dir:
#     if os.path.basename(file)[:4]==year:
#         fn = os.path.basename(file)
#         save_bad.append(fn)

In [ ]:
# Plot 'not_good' scenes
not_good = f'/home/etboud/projects/data/rerun/{name}/V5/not_good/'
dir = glob.glob(not_good + '/*.tif')
for file in dir:
    fn = os.path.basename(file)
    date = fn.split('_')[0]
    id = fn.split('_')[1]
    code = f'{date}_{id}'

    for raw_file in ps_raw_subdir:
        if code in raw_file:
            matching_raw_file = glob.glob(raw_file+'/*/PSScene/*_SR_clip.tif')[0]
            rgb_image = rioxarray.open_rasterio(matching_raw_file,masked = True, drop=True)
            rgb_image = rgb_image.rio.clip(basin.geometry, basin.crs)
            _, _, _, _, rgb_image = calc_rgb(rgb_image)
            
            sca = rioxarray.open_rasterio(file, all_touched=False, drop=True, masked=True)
            sca.values = np.where(np.isnan(sca.values), 0, sca.values)
            sca = sca.rio.clip(basin.geometry.values, crs=basin.crs, drop=True)
            sca = np.squeeze(sca.values)
            
            plt.subplot(1,2,1)
            plt.imshow(sca,vmin= 0,vmax = 2, interpolation= 'none')
            plt.colorbar()
            plt.title(code)
            plt.axis('off')
            
            plt.subplot(1,2,2)
            plt.imshow(rgb_image)
            plt.axis('off')
            plt.show()

### Determining NOT GOOD

In [ ]:
# Plot  scenes
# File path to processed SCA which no scenes have been filitered out
file_path = f'/home/etboud/projects/data/rerun/{name}/V5/'
dir = glob.glob(file_path + '/*.tif')
for file in dir:
    fn = os.path.basename(file)
    date = fn.split('_')[0]
    id = fn.split('_')[1]
    code = f'{date}_{id}'

    for raw_file in ps_raw_subdir:
        if code in raw_file:
            matching_raw_file = glob.glob(raw_file+'/*/PSScene/*_SR_clip.tif')[0]
            rgb_image = rioxarray.open_rasterio(matching_raw_file,masked = True, drop=True)
            rgb_image = rgb_image.rio.clip(basin.geometry, basin.crs)
            _, _, _, _, rgb_image = calc_rgb(rgb_image)
            
            sca = rioxarray.open_rasterio(file, all_touched=False, drop=True, masked=True)
            sca.values = np.where(np.isnan(sca.values), 0, sca.values)
            sca = sca.rio.clip(basin.geometry.values, crs=basin.crs, drop=True)
            sca = np.squeeze(sca.values)
            
            plt.subplot(1,2,1)
            plt.imshow(sca,vmin= 0,vmax = 2, interpolation= 'none')
            plt.colorbar()
            plt.title(code)
            plt.axis('off')
            
            plt.subplot(1,2,2)
            plt.imshow(rgb_image)
            plt.axis('off')
            plt.show()

### IF moving raw images

In [ ]:
# #moving raw subdirectories for bad PS images to a different directory
# path = f'/data0/images/planet/emma/planet/{name}/'
# move_to = f'/data0/images/planet/emma/planet/{name}/not_good/'
# dir = glob.glob(path + '/*.tif')
# save_bad_codes = [fn.split('_')[0] + '_' + fn.split('_')[1] for fn in save_bad]

# for file in ps_raw_subdir:
#     fn = os.path.basename(file)
#     date = fn.split('_')[0]
#     id = fn.split('_')[1]
#     code = f'{date}_{id}'
#     subdir = file.split('/')[-1]
#     if code in save_bad_codes:
#         src = path+subdir
#         dst = move_to+subdir
#         shutil.move(src, dst)
#         # print(subdir)

## Save corresponding files

In [34]:
def closest_dates(ps_dates, aso_date):
    # Extract dates from tuples
    ps_date_list = [date for date, _ in ps_dates]
    # Sort the ps_dates
    sorted_list = sorted(ps_date_list)
    
    closest_distance = None
    closest_dates = []

    for date in sorted_list:
        distance = abs((date - aso_date).days)
        
        if closest_distance is None or distance < closest_distance:
            closest_distance = distance
            closest_dates = [date]
        elif distance == closest_distance:
            closest_dates.append(date)

    return closest_dates

In [17]:
name

'DLNY'

In [39]:
# create dataframe to store all PS filenames with corresponding ASO flights
raw_fn = []
for aso_date, aso_file in sorted(aso_dates):
    closest = closest_dates(ps_dates, aso_date)

    closest_str = closest[0].strftime('%Y%m%d') # change index fromo to -1 to get the preceeding or following closest date when dates adjacent are equadistance
    # ps_dates = pd.to_datetime(closest_str, format='%Y-%m-%d') 
    print(closest_str)
    for fn in ps_sca_tif:
        if closest_str in fn:
            matching_model_files = glob.glob(fn)
            code = fn.split('/')[-1].split('_')[1]
            closest_code = f'{closest_str}_{code}'
    for file in ps_raw_subdir:
        if closest_code in file:
            matching_raw_files = glob.glob(file+'/*/PSScene/*_SR_clip.tif')[0]
            raw_fn.append({"date": closest_str, "raw_file": matching_raw_files,"model_file": matching_model_files, "aso_file": aso_file,"aso_date": aso_date})
            
pd_dlny_0 = pd.DataFrame(raw_fn)

20220405
20220429
20220519
20230426
20230601
20230629


In [40]:
merged = pd.concat([pd_dlny_0,pd_dlny_1,pd_budd_0,pd_budd_1], ignore_index=True) #run in different cell after creating multiple DF for each year

In [41]:
def clean_string(x):
    if isinstance(x, list):
        # Convert list to string, then clean it
        x = ', '.join(map(str, x))
    elif not isinstance(x, str):
        return x  # Return as is if not a string or list

    # Now x is guaranteed to be a string, so we can clean it
    return x.replace('[', '').replace(']', '').replace("'", '')

# Apply the cleaning function to both columns
merged['raw_file'] = merged['raw_file'].apply(clean_string)
merged['model_file'] = merged['model_file'].apply(clean_string)

# Format date columns
merged['ps_date'] = pd.to_datetime(merged['date']).dt.strftime('%Y-%m-%d')
merged['aso_date'] = pd.to_datetime(merged['aso_date']).dt.strftime('%Y-%m-%d')
merged['model_run'] = merged['model_file'].str.split('/').str[-2]
# Drop the original date column
merged.drop(columns=['date'], inplace=True)
merged_drop = merged.drop_duplicates()
# merged_drop.to_csv('corresponding_files.csv', index=False)

In [ ]:
# Loop through the corresponding files
for index, row in merged_drop.iterrows():
    raw_fn = row['raw_file']
    model_fn = row['model_file']
    aso_fn = row['aso_file']
    
    fn = raw_fn.split('/')[-1].split('_')
    id = f'{fn[0]}_{fn[1]}'
    name = model_fn.split('/')[-3]
    basin = gpd.read_file(f'/home/etboud/projects/data/basins/{name}/{name}_4326.geojson')
    basin = basin.to_crs('EPSG:32611')
    rgb_image = rxr.open_rasterio(raw_fn,masked = True, drop=True)
    rgb_image = rgb_image.rio.clip(basin.geometry, basin.crs)
    _, _, _, _, rgb_image = calc_rgb(rgb_image)

    
    model_image = rxr.open_rasterio(model_fn, masked=True, drop=True)
    model_image.values = np.where(np.isnan(model_image.values), 0, model_image.values)
    model_image = model_image.rio.clip(basin.geometry, basin.crs)
    
    aso = rxr.open_rasterio(aso_fn, masked=True)
    
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 4, 1)
    plt.imshow(rgb_image)
    plt.title(f'RGB Image {id}')
    plt.axis('off')

    # Original Image
    plt.subplot(1, 4, 2)
    plt.imshow(model_image[0], interpolation ='none', cmap='Blues',vmin = 0, vmax = 1)
    plt.title('Model Output')
    plt.axis('off')


    # Convolved Output
    plt.subplot(1, 4, 3)
    plt.imshow(aso[0], interpolation='none', cmap='Blues',vmin = 0,vmax=1)
    plt.title(row['aso_date'])
    plt.axis('off')

    plt.tight_layout()
    plt.show()
    
    

In [43]:
corr_files = pd.read_csv('corresponding_files.csv')

In [44]:
corr_files

,raw_file,model_file,aso_file,aso_date,ps_date,model_run
0,/data0/images/planet/emma/planet/DLNY/20220405...,/home/etboud/projects/data/rerun/DLNY/V7/20220...,/home/etboud/projects/data/aso/validation/DLNY...,2022-04-05,2022-04-05,V7
1,/data0/images/planet/emma/planet/DLNY/20220429...,/home/etboud/projects/data/rerun/DLNY/V7/20220...,/home/etboud/projects/data/aso/validation/DLNY...,2022-04-29,2022-04-29,V7
2,/data0/images/planet/emma/planet/DLNY/20220519...,/home/etboud/projects/data/rerun/DLNY/V7/20220...,/home/etboud/projects/data/aso/validation/DLNY...,2022-05-18,2022-05-19,V7
3,/data0/images/planet/emma/planet/DLNY/20230426...,/home/etboud/projects/data/rerun/DLNY/V7/20230...,/home/etboud/projects/data/aso/validation/DLNY...,2023-04-27,2023-04-26,V7
4,/data0/images/planet/emma/planet/DLNY/20230601...,/home/etboud/projects/data/rerun/DLNY/V7/20230...,/home/etboud/projects/data/aso/validation/DLNY...,2023-06-01,2023-06-01,V7
5,/data0/images/planet/emma/planet/DLNY/20230629...,/home/etboud/projects/data/rerun/DLNY/V7/20230...,/home/etboud/projects/data/aso/validation/DLNY...,2023-06-26,2023-06-29,V7
6,/data0/images/planet/emma/planet/BUDD/20220403...,/home/etboud/projects/data/rerun/BUDD/V7/20220...,/home/etboud/projects/data/aso/validation/BUDD...,2022-04-05,2022-04-03,V7
7,/data0/images/planet/emma/planet/BUDD/20220430...,/home/etboud/projects/data/rerun/BUDD/V7/20220...,/home/etboud/projects/data/aso/validation/BUDD...,2022-04-29,2022-04-30,V7
8,/data0/images/planet/emma/planet/BUDD/20220518...,/home/etboud/projects/data/rerun/BUDD/V7/20220...,/home/etboud/projects/data/aso/validation/BUDD...,2022-05-18,2022-05-18,V7
9,/data0/images/planet/emma/planet/BUDD/20230426...,/home/etboud/projects/data/rerun/BUDD/V7/20230...,/home/etboud/projects/data/aso/validation/BUDD...,2023-04-27,2023-04-26,V7


In [45]:
update = pd.concat([corr_files,merged_drop], ignore_index=True)

In [46]:
update

,raw_file,model_file,aso_file,aso_date,ps_date,model_run
0,/data0/images/planet/emma/planet/DLNY/20220405...,/home/etboud/projects/data/rerun/DLNY/V7/20220...,/home/etboud/projects/data/aso/validation/DLNY...,2022-04-05,2022-04-05,V7
1,/data0/images/planet/emma/planet/DLNY/20220429...,/home/etboud/projects/data/rerun/DLNY/V7/20220...,/home/etboud/projects/data/aso/validation/DLNY...,2022-04-29,2022-04-29,V7
2,/data0/images/planet/emma/planet/DLNY/20220519...,/home/etboud/projects/data/rerun/DLNY/V7/20220...,/home/etboud/projects/data/aso/validation/DLNY...,2022-05-18,2022-05-19,V7
3,/data0/images/planet/emma/planet/DLNY/20230426...,/home/etboud/projects/data/rerun/DLNY/V7/20230...,/home/etboud/projects/data/aso/validation/DLNY...,2023-04-27,2023-04-26,V7
4,/data0/images/planet/emma/planet/DLNY/20230601...,/home/etboud/projects/data/rerun/DLNY/V7/20230...,/home/etboud/projects/data/aso/validation/DLNY...,2023-06-01,2023-06-01,V7
5,/data0/images/planet/emma/planet/DLNY/20230629...,/home/etboud/projects/data/rerun/DLNY/V7/20230...,/home/etboud/projects/data/aso/validation/DLNY...,2023-06-26,2023-06-29,V7
6,/data0/images/planet/emma/planet/BUDD/20220403...,/home/etboud/projects/data/rerun/BUDD/V7/20220...,/home/etboud/projects/data/aso/validation/BUDD...,2022-04-05,2022-04-03,V7
7,/data0/images/planet/emma/planet/BUDD/20220430...,/home/etboud/projects/data/rerun/BUDD/V7/20220...,/home/etboud/projects/data/aso/validation/BUDD...,2022-04-29,2022-04-30,V7
8,/data0/images/planet/emma/planet/BUDD/20220518...,/home/etboud/projects/data/rerun/BUDD/V7/20220...,/home/etboud/projects/data/aso/validation/BUDD...,2022-05-18,2022-05-18,V7
9,/data0/images/planet/emma/planet/BUDD/20230426...,/home/etboud/projects/data/rerun/BUDD/V7/20230...,/home/etboud/projects/data/aso/validation/BUDD...,2023-04-27,2023-04-26,V7


In [47]:
update.to_csv('corresponding_files.csv', index=False)